# Parquet konvertering-, partisjonering-, filtrering og visualiseringsverktøy

Dette verktøyet svarer på brukerhistorien:

"Som havnesjef i Kristiansand kommune ønsker jeg å vite hvilke skip som befant seg innen en viss radius fra Kristiansand sentrum på en bestemt dato."


## Hvordan bruke dette verktøyet
1. Installer alle nødvendige pakker
2. Importer pakkene inn i verktøyet
3. Kjør cellen med funksjonsdefinisjonene
4. Opprett mappestruktur
5. Last opp parquet-filer til mappen `data/raw`
6. Start konvertering og partisjonering – de konverterte filene lagres i mappen `data/processed`
7. Hent og visualiser skip

## Støttede filformater

- Parquet-filer med geografisk informasjon


## 1. Last inn nødvendige pakker

Kjør cellen nedenfor for å importere pakkene som verktøyet trenger:

In [ ]:
!pip install -r requirements.txt

## 2. Importer pakkene inn i verktøyet

In [1]:
import os
import time
import pyproj
import datetime
from datetime import datetime
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely import wkb
import folium
from folium import plugins
from shapely.geometry import Point, Polygon
from shapely.geometry import Point
from shapely.ops import transform
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Globale variabler
_current_observer = None
global_paths = None
notebook_observer = None
data_dir = "./data"  # Standard datamappe


## 3. Definer funksjoner og globale variabler
Kjør cellen nedenfor for å definere funksjonene som konverterer filene:

In [40]:
# Global variabel for mappestier
_current_observer = None
global_paths = None

# Sett standardverdier for parameterne
data_dir = "./data"
num_files = 5

def create_folders(data_folder="./data"):
    """
    Oppretter nødvendig mappestruktur for konvertering.

    Args:
        data_folder: Sti til hovedmappen for data

    Returns:
        dict: Stier til opprettede mapper
    """
    # Definer mappestruktur
    raw_folder = os.path.join(data_folder, "raw")
    processed_folder = os.path.join(data_folder, "processed")
    source_folder = os.path.join(data_folder, "source")
    incoming_folder = os.path.join(data_folder, "incoming")

    # Opprett mapper
    os.makedirs(data_folder, exist_ok=True)
    os.makedirs(raw_folder, exist_ok=True)
    os.makedirs(processed_folder, exist_ok=True)
    os.makedirs(source_folder, exist_ok=True)
    os.makedirs(incoming_folder, exist_ok=True)

    # Returner stier for senere bruk
    return {
        "raw_folder": os.path.abspath(data_folder),
        "processed_folder": os.path.abspath(raw_folder),
        "prosessert_mappe": os.path.abspath(processed_folder),
        "source_folder": os.path.abspath(source_folder),
        "incoming_folder": os.path.abspath(incoming_folder)
    }

def add_time_partitioning_columns(df, time_column='date_time_utc'):
    """
    Legger til kolonner for tidspartisjonering (kun time for partisjonering, men beholder dato-kolonner).
    """
    if time_column not in df.columns:
        print(f"Advarsel: Tidsstempelkolonne '{time_column}' finnes ikke")
        return df

    if not pd.api.types.is_datetime64_any_dtype(df[time_column]):
        print(f"Advarsel: Kolonnen '{time_column}' er ikke en datetime-kolonne")
        return df

    df_with_time = df.copy()
    df_with_time['year'] = df_with_time[time_column].dt.year
    df_with_time['month'] = df_with_time[time_column].dt.month
    df_with_time['day'] = df_with_time[time_column].dt.day
    df_with_time['hour'] = df_with_time[time_column].dt.hour

    return df_with_time

def create_geodataframe(df):
    """
    Oppretter en GeoDataFrame fra en DataFrame ved å finne koordinater eller geometrikolonner.
    """

    # Sjekk for lat/long kolonner
    if 'longitude' in df.columns and 'latitude' in df.columns:
        try:
            return gpd.GeoDataFrame(
                df,
                geometry=gpd.points_from_xy(df.longitude, df.latitude),
                crs="EPSG:4326"
            )
        except Exception as e:
            print(f"Kunne ikke opprette geometri fra lat/long: {e}")

    # Sjekk for andre geometrikolonner
    geom_columns = [col for col in df.columns if any(
        term in col.lower() for term in ['geom', 'coord', 'point', 'polygon', 'linestring', 'wkt']
    )]

    for col in geom_columns:
        if df[col].dtype != 'object':
            continue

        try:
            geom = df[col].apply(wkt.loads)
            return gpd.GeoDataFrame(df, geometry=geom, crs="EPSG:4326")
        except Exception:
            continue

    return None

def save_partitioned_geoparquet(gdf, output_path, partition_columns):
    """
    Lagrer en GeoDataFrame som partisjonert GeoParquet.
    """

    # Konverterer geometri til WKB for å kunne partisjonere
    df_med_wkb = gdf.copy()
    df_med_wkb['geometry_wkb'] = df_med_wkb['geometry'].apply(lambda geom: wkb.dumps(geom))
    df_for_partisjon = df_med_wkb.drop(columns=['geometry'])

    # Utfør partisjonering
    df_for_partisjon.to_parquet(output_path, partition_cols=partition_columns)

    # Konverter hver partisjonert fil tilbake til GeoParquet
    convert_partitioned_files_to_geoparquet(output_path)
    return output_path

def create_year_folder(year, output_path):
    """Oppretter en mappe for et spesifikt år."""
    year_folder = os.path.join(output_path, f"year={year}")
    os.makedirs(year_folder, exist_ok=True)
    return year_folder

def prepare_wkb_dataframe(year_df):
    """Forbereder en dataframe med WKB-konvertert geometri for partisjonering."""
    # Konverter geometri til WKB for å kunne partisjonere
    year_df_wkb = year_df.copy()
    year_df_wkb['geometry_wkb'] = year_df_wkb['geometry'].apply(lambda geom: wkb.dumps(geom))
    year_df_wkb = year_df_wkb.drop(columns=['geometry'])
    return year_df_wkb

def convert_file_to_geoparquet(src_file, dst_file):
    """Konverterer en enkelt fil fra WKB-format til GeoParquet."""
    try:
        part_df = pd.read_parquet(src_file)
        if 'geometry_wkb' not in part_df.columns:
            return False

        part_df['geometry'] = part_df['geometry_wkb'].apply(lambda x: wkb.loads(x))
        part_df = part_df.drop(columns=['geometry_wkb'])
        part_gdf = gpd.GeoDataFrame(part_df, geometry='geometry', crs="EPSG:4326")

        # Sørg for at målmappen eksisterer
        os.makedirs(os.path.dirname(dst_file), exist_ok=True)
        part_gdf.to_parquet(dst_file)
        return True
    except Exception as e:
        print(f"Advarsel: Kunne ikke konvertere {src_file} til GeoParquet: {e}")
        return False

def copy_and_convert_files(temp_year_folder, year_folder):
    """Kopierer og konverterer filer fra temp-mappen til målmappen."""
    success_count = 0
    error_count = 0

    # Opprett først alle mappene
    for root, dirs, _ in os.walk(temp_year_folder):
        for directory in dirs:
            src_dir = os.path.join(root, directory)
            rel_path = os.path.relpath(src_dir, temp_year_folder)
            dst_dir = os.path.join(year_folder, rel_path)
            os.makedirs(dst_dir, exist_ok=True)

    # Kopier og konverter filene
    for root, _, files in os.walk(temp_year_folder):
        for file in files:
            src_file = os.path.join(root, file)
            rel_path = os.path.relpath(src_file, temp_year_folder)
            dst_file = os.path.join(year_folder, rel_path)

            if convert_file_to_geoparquet(src_file, dst_file):
                success_count += 1
            else:
                error_count += 1

    return success_count, error_count

def convert_partitioned_files_to_geoparquet(root_folder):
    """
    Konverterer alle partisjonerte parquet-filer til GeoParquet format.
    """
    error_count = 0

    for root, _, files in os.walk(root_folder):
        for file in files:
            if not file.endswith('.parquet'):
                continue

            parquet_path = os.path.join(root, file)
            try:
                # Les dataframe
                part_df = pd.read_parquet(parquet_path)

                # Hopp over hvis den ikke har geometry_wkb
                if 'geometry_wkb' not in part_df.columns:
                    continue

                # Konverter WKB tilbake til geometri
                part_df['geometry'] = part_df['geometry_wkb'].apply(lambda x: wkb.loads(x))
                part_df = part_df.drop(columns=['geometry_wkb'])

                # Lag GeoDataFrame
                part_gdf = gpd.GeoDataFrame(part_df, geometry='geometry', crs="EPSG:4326")

                # Skriv GeoParquet-filen
                part_gdf.to_parquet(parquet_path)
            except Exception as e:
                print(f"Feil ved konvertering av {parquet_path}: {e}")
                error_count += 1

    if error_count > 0:
        print(f"Advarsel: {error_count} filer kunne ikke konverteres til GeoParquet")

def convert_parquet_to_geoparquet(file_path, output_path, partition_columns=None):
    """
    Konverterer parquet-fil til GeoParquet-format med partisjonering.
    Args:
        file_path: Sti til parquet-filen
        output_path: Sti hvor GeoParquet-filen skal lagres
        partition_columns: Liste av kolonnenavn som skal brukes for partisjonering
    """
    try:
        df = pd.read_parquet(file_path)
    except Exception as e:
        print(f"Kunne ikke lese parquet-fil: {e}")
        return False

    # Håndter tidspartisjonering
    time_partitioning = False
    if partition_columns and 'date_time_utc' in partition_columns:
        partition_columns.remove('date_time_utc')
        time_partitioning = True

    # Legg til alle tidspartisjoneringskolonner, men vi vil bare partisjonere på time
    if time_partitioning:
        df = add_time_partitioning_columns(df)
        # Sett opp partisjonering kun på 'hour'
        partition_columns = ['hour'] + (partition_columns or [])

    # Opprett GeoDataFrame
    gdf = create_geodataframe(df)
    if gdf is None:
        return False

    # Sjekk at alle partisjoneringskolonner finnes
    if partition_columns and not all(col in gdf.columns for col in partition_columns):
        print(f"Advarsel: Ikke alle partisjoneringskolonner finnes i datasettet")
        missing = [col for col in partition_columns if col not in gdf.columns]
        print(f"Manglende kolonner: {missing}")
        return False

    # Lagre med partisjonering (bare time)
    if partition_columns:
        return save_partitioned_geoparquet(gdf, output_path, partition_columns)
    else:
        gdf.to_parquet(output_path)
        return output_path

def convert_all_parquet_files(data_folder, partition_columns=None):
    """
    Konverterer alle parquet-filer i en mappe til GeoParquet.
    """
    raw_folder = os.path.join(data_folder, "raw")
    processed_folder = os.path.join(data_folder, "processed")

    # Opprett mapper
    os.makedirs(raw_folder, exist_ok=True)
    os.makedirs(processed_folder, exist_ok=True)

    results = {
        "converted": [],
        "failed": []
    }

    # Finn alle parquet-filer
    parquet_files = [f for f in os.listdir(raw_folder)
                     if f.lower().endswith('.parquet') and os.path.isfile(os.path.join(raw_folder, f))]

    if not parquet_files:
        print("Ingen parquet-filer funnet i råmappen")
        return results

    # Konverter hver fil
    for filename in parquet_files:
        file_path = os.path.join(raw_folder, filename)
        base_filename = os.path.splitext(filename)[0]
        output_path = os.path.join(processed_folder, f"{base_filename}.parquet")

        # Konverter filen
        resultat = convert_parquet_to_geoparquet(
            file_path,
            output_path,
            partition_columns.copy() if partition_columns else None,
        )

        if resultat:
            results["converted"].append(file_path)
        else:
            results["failed"].append(file_path)
            print(f"Kunne ikke konvertere: {file_path}")

    return results

def start_conversion(data_folder="./data", partition_columns=None):
    """
    Start konverteringsprosessen fra Parquet til GeoParquet.
    Args:
        data_folder: Sti til datamappen
        partition_columns: Liste av kolonner for partisjonering
    """
    start_time = time.time()
    print(f"Starter konvertering og partisjonering av filer ...\n")
    print(f"Valgte kolonner: {partition_columns}.")
    print("Vent litt ...")

    # Utfør konverteringen
    results = convert_all_parquet_files(data_folder, partition_columns)

    print("\nKonvertering og partisjonering utført:")
    print(f"• Total behandlingstid: {time.time() - start_time:.2f} sekunder")
    print(f"• Konverterte filer: {len(results['converted'])}")

    return results

def define_agder_polygon():
    """
    Definerer et forenklet polygon for Agder-kysten.

    Returns:
        Shapely Polygon som definerer Agder-kystlinjen
    """

    agder_coords = [
        (6.8, 57.8),  # Sørvest
        (10.0, 57.8),  # Sørøst
        (10.0, 59.5),  # Nordøst
        (6.8, 59.5),  # Nordvest
        (6.8, 57.8)   # Lukk polygonet
    ]
    return Polygon(agder_coords)

def filter_ais_data(root_folder, date=None, filters=None, agder_polygon=None, max_number_of_ships=10):
    """
    Filtrerer AIS-data basert på dato og geografisk område.

    Args:
        root_folder: Mappen hvor dataene er lagret
        date: Dato for filtrering
        filters: Dictionary med kolonnenavn og verdier for filtrering
        agder_polygon: Polygon som definerer Agder-kysten
        max_number_of_ships: Maksimalt antall skip å returnere

    Returns:
        GeoDataFrame med filtrerte skip
    """
    if not date:
        print("Dato er påkrevd")
        return None

    if filters is None:
        filters = {}

    # Dato-komponenter for filtrering senere
    year, month, day = date.year, date.month, date.day

    # Konverter dato til string for bruk i filnavn
    dato_str = date.strftime('%Y-%m-%d')


    # Sjekk om rot-mappen eksisterer
    if not os.path.exists(root_folder):
        print(f"Mappen {root_folder} finnes ikke")
        return None

    # Finn alle mapper i rot-mappen
    date_folders = []
    for element in os.listdir(root_folder):
        full_path = os.path.join(root_folder, element)
        if os.path.isdir(full_path):
            date_folders.append(element)

    if not date_folders:
        print(f"Ingen datamapper funnet i {root_folder}")
        return None

    # Finn alle parquet-filer rekursivt
    date_found = []

    for date_folder in date_folders:
        base_path = os.path.join(root_folder, date_folder)
        # Traverser mappestrukturen rekursivt
        for root, dirs, files in os.walk(base_path):
            # For time-partisjonering, sjekk fil-innhold istedenfor å basere på katalogstruktur
            for file in files:
                if file.endswith('.parquet'):
                    file_path = os.path.join(root, file)
                    date_found.append(file_path)

    if not date_found:
        print(f"Ingen parquet-filer funnet")
        return None

    # Les inn og filtrer dataene
    dfs = []
    for file_path in date_found:
        try:
            # Ekstraher partisjonsverdier fra stien (for time)
            time_value = None
            parts = file_path.split(os.sep)
            for part in parts:
                if part.startswith('hour='):
                    time_value = int(part.split('=')[1])

            # Les parquet-filen
            df = pd.read_parquet(file_path)

            # Legg til hour som kolonne hvis den ikke allerede finnes
            if time_value is not None and 'hour' not in df.columns:
                df['hour'] = time_value

            # Filtrer på dato
            if 'year' in df.columns and 'month' in df.columns and 'day' in df.columns:
                df = df[(df['year'] == year) & (df['month'] == month) & (df['day'] == day)]

                if df.empty:
                    # Hopp over denne filen hvis ingen rader matcher datoen
                    continue

            # Utfør ekstra filtrering basert på filtre-parameteren
            ship_df = False
            for columns, value in filters.items():
                if columns not in df.columns:
                    ship_df = True
                    break

                # For alle kolonner, filtrer på vanlig måte
                if isinstance(value, list):
                    if not df[columns].isin(value).any():
                        ship_df = True
                        break
                else:
                    if not (df[columns] == value).any():
                        ship_df = True
                        break

            if not ship_df:
                # Gjør ytterligere filtrering på dataframe-nivå
                for columns, value in filters.items():
                    if columns in df.columns:
                        if isinstance(value, list):
                            df = df[df[columns].isin(value)]
                        else:
                            df = df[df[columns] == value]

                if not df.empty:
                    dfs.append(df)

        except Exception as e:
            print(f"Feil ved lesing av {file_path}: {str(e)}")

    if not dfs:
        print("Ingen data lest inn")
        return None

    # Slå sammen alle dataframes
    combined_df = pd.concat(dfs, ignore_index=True)

    # Konverter til GeoDataFrame hvis koordinater finnes
    if 'longitude' in combined_df.columns and 'latitude' in combined_df.columns:
        # Opprett geometrikolonne fra longitude og latitude
        geometry = [Point(xy) for xy in zip(combined_df['longitude'], combined_df['latitude'])]
        geo_df = gpd.GeoDataFrame(combined_df, geometry=geometry, crs="EPSG:4326")

        # Filtrer basert på Agder-polygon hvis gitt
        if agder_polygon is not None:
            # Sjekk om agder_polygon er et GeoDataFrame eller et Shapely-objekt
            if isinstance(agder_polygon, gpd.GeoDataFrame):
                # Sørg for at CRS er likt
                if agder_polygon.crs != geo_df.crs:
                    agder_polygon = agder_polygon.to_crs(geo_df.crs)

                # Spatial join - dette kan ta litt tid for store datasett
                geo_df = gpd.sjoin(geo_df, agder_polygon, how="inner", predicate="within")
            else:
                # Antar at agder_polygon er et Shapely-objekt
                geo_df = geo_df[geo_df.geometry.within(agder_polygon)]

        # Sorter etter nyeste data først for hvert skip
        if 'timestamp' in geo_df.columns:
            geo_df = geo_df.sort_values(by='timestamp', ascending=False)

        # Begrens til unike skip
        if 'mmsi' in geo_df.columns or 'ship_name' in geo_df.columns:
            id_columns = 'mmsi' if 'mmsi' in geo_df.columns else 'ship_name'
            unique_ships = geo_df[id_columns].unique()

            if len(unique_ships) > max_number_of_ships:
                print(f"Begrenser til {max_number_of_ships} skip")
                unique_ships = unique_ships[:max_number_of_ships]

            geo_df = geo_df[geo_df[id_columns].isin(unique_ships)]

            # Hent første rad for hvert unikt skip for å få en kompakt liste
            geo_df = geo_df.drop_duplicates(subset=[id_columns])

        return geo_df
    else:
        print("Mangler koordinater (longitude/latitude)")
        return None

def find_ships_near_point(ship_gdf, point, buffer_distance=500):
    """
    Filtrerer skip som er innenfor en gitt avstand fra et punkt.

    Args:
        ship_gdf: GeoDataFrame med skip
        point: Tuple (longitude, latitude)
        buffer_distance: Avstand i meter

    Returns:
        GeoDataFrame med filtrerte skip
    """
    if ship_gdf is None or ship_gdf.empty:
        return None, None

    try:
        # Opprett punkt fra koordinater
        center_point = Point(point)

        # Projiser til et lokalt koordinatsystem som bruker meter som enheter (UTM-sone)
        # Finn UTM-sone basert på longitude
        utm_sone = int((point[0] + 180) / 6) + 1
        utm_crs = f"EPSG:326{utm_sone}"  # Nordlig halvkule

        # Transformer funksjonen for å lage buffer i meter
        project_to_utm = pyproj.Transformer.from_crs(
            "EPSG:4326", utm_crs, always_xy=True).transform
        project_to_wgs84 = pyproj.Transformer.from_crs(
            utm_crs, "EPSG:4326", always_xy=True).transform

        # Projiser punkt til UTM, lag buffer i meter, projiser tilbake til WGS84
        utm_point = transform(project_to_utm, center_point)
        buffer_utm = utm_point.buffer(buffer_distance)
        buffer_wgs84 = transform(project_to_wgs84, buffer_utm)

        # Filtrer for skip innenfor bufferen
        ships_within_buffer = ship_gdf[ship_gdf.geometry.within(buffer_wgs84)]

        return ships_within_buffer, buffer_wgs84

    except Exception as e:
        print(f"Feil ved buffering: {str(e)}")
        return None, None

def filter_ships(root_folder, dato, radius=None, center_point=(8.0182, 58.1599), number_of_ships=10):
    """
    Filtrerer skip i Agder-området, enten basert på et definert polygon eller en radius fra et senterpunkt.

    Args:
        root_folder: Mappen hvor de konverterte dataene er lagret
        dato: Datoen å filtrere på (datetime objekt)
        radius: Radius i meter fra senterpunkt (None = bruk polygon)
        center_point: Tuple (longitude, latitude) for senterpunkt (standard: Kristiansand)
        number_of_ships: Maksimalt antall skip å returnere

    Returns:
        GeoDataFrame med filtrerte skip
    """
    # Først hent alle skip for datoen uten geografisk filtrering
    ships_on_date = filter_ais_data(
        root_folder=root_folder,
        date=dato,
        filters=None,
        agder_polygon=None,  # Ingen geografisk filter her ennå
        max_number_of_ships=100  # Hent litt flere for å kunne filtrere geografisk etterpå
    )

    if ships_on_date is None or ships_on_date.empty:
        print(f"Ingen skip funnet på datoen {dato.strftime('%d.%m.%Y')}")
        return None

    # Hvis radius er oppgitt, bruk finn_skip_rundt_punkt
    if radius is not None and radius > 0:
        #print(f"Søker etter skip innen {radius} meter fra senterpunkt")
        ships_near_point, buffer = find_ships_near_point(
            ships_on_date,
            center_point,
            buffer_distance=radius
        )

        # Begrens antall skip hvis nødvendig
        if ships_near_point is not None and len(ships_near_point) > number_of_ships:
            ships_near_point = ships_near_point.head(number_of_ships)

        if ships_near_point is None or ships_near_point.empty:
            print(f"Ingen skip funnet innenfor {radius}m fra senterpunkt på datoen {dato.strftime('%d.%m.%Y')}")
            return None

        #print(f"Fant {len(ships_near_point)} skip innenfor {radius}m på datoen {dato.strftime('%d.%m.%Y')}")
        return ships_near_point
    else:
        # Bruk standard Agder-polygon
        agder_polygon = define_agder_polygon()

        # Filtrer med polygon
        ships_within_agder = ships_on_date[ships_on_date.geometry.within(agder_polygon)]

        # Begrens antall skip
        if len(ships_within_agder) > number_of_ships:
            ships_within_agder = ships_within_agder.head(number_of_ships)

        if ships_within_agder.empty:
            print(f"Ingen skip funnet innenfor Agder-område på datoen {dato.strftime('%d.%m.%Y')}")
            return None

        #print(f"Fant {len(ships_within_agder)} skip innenfor Agderkysten på datoen {dato.strftime('%d.%m.%Y')}")
        return ships_within_agder

def visualize_ships(ship_gdf, agder_polygon=None, tittel="Skip innenfor Agder"):
    """
    Visualiserer skip på et interaktivt kart med Folium.

    Args:
        ship_gdf: GeoDataFrame med skip
        agder_polygon: GeoDataFrame eller Shapely polygon med Agder-området (valgfri)
        tittel: Tittel på kartet

    Returns:
        Folium Map objekt
    """
    if ship_gdf is None or ship_gdf.empty:
        print("Ingen skip å visualisere")
        return None

    # Beregn senterpunkt for kartet basert på skipene
    center_point = [ship_gdf.geometry.y.mean(), ship_gdf.geometry.x.mean()]

    # Opprett kartet
    m = folium.Map(
        location=center_point,
        zoom_start=9,
        tiles='OpenStreetMap',
        control_scale=True
    )

    # Legg til Agder-polygon hvis tilgjengelig
    if agder_polygon is not None:
        if isinstance(agder_polygon, gpd.GeoDataFrame):
            # Hvis det er en GeoDataFrame, konverter til GeoJSON og legg til
            folium.GeoJson(
                data=agder_polygon,
                name="Agder fylke",
                style_function=lambda x: {
                    'fillColor': '#aaaaff',
                    'color': '#0000ff',
                    'weight': 2,
                    'fillOpacity': 0.1
                }
            ).add_to(m)
        else:
            # Hvis det er et Shapely polygon, konverter koordinater og legg til
            if isinstance(agder_polygon, Polygon):
                coords = list(agder_polygon.exterior.coords)
                # Konverter til format [lat, lon] som Folium forventer
                folium_coords = [[y, x] for x, y in coords]

                folium.Polygon(
                    locations=folium_coords,
                    color='blue',
                    weight=2,
                    fill=True,
                    fill_color='#aaaaff',
                    fill_opacity=0.1,
                    name='Agder fylke'
                ).add_to(m)

    # Opprett en MarkerCluster for skipene
    marker_cluster = plugins.MarkerCluster(name="Skip").add_to(m)

    # Legg til markører for hvert skip
    for idx, row in ship_gdf.iterrows():
        # Samle informasjon for popup
        popup_info = "<table>"
        for col in ship_gdf.columns:
            if col != 'geometry' and pd.notna(row[col]):
                popup_info += f"<tr><th>{col}</th><td>{row[col]}</td></tr>"
        popup_info += "</table>"

        # Opprett popup
        popup = folium.Popup(popup_info, max_width=300)

        # Hent skipnavn
        ship_name = row.get('ship_name', f"Skip {idx}")

        # Opprett ikon
        icon = folium.Icon(icon='ship', prefix='fa', color='blue')

        # Legg til markør
        folium.Marker(
            location=[row.geometry.y, row.geometry.x],
            popup=popup,
            tooltip=ship_name,
            icon=icon
        ).add_to(marker_cluster)

    # Legg til fullskjermsknapp
    plugins.Fullscreen().add_to(m)

    # Legg til lagkontroll (for å slå av/på lag)
    folium.LayerControl().add_to(m)

    # Legg til tegneforklaring
    legend_html = """
    <div style="position: fixed; bottom: 50px; left: 50px; z-index: 1000;
                background-color: white; padding: 10px; border-radius: 5px;
                border: 2px solid grey; font-size: 14px;">
      <p><b>Tegnforklaring</b></p>
      <p>
        <i class="fa fa-ship fa-1x" style="color:blue"></i> Skip<br>
        <span style="background-color: #aaaaff; opacity: 0.1; border: 1px solid blue;">&nbsp;&nbsp;&nbsp;&nbsp;</span> Agder fylke
      </p>
    </div>
    """
    m.get_root().html.add_child(folium.Element(legend_html))

    #m.fit_bounds(m.get_bounds())

    return m

def filter_ships_in_agder(rot_mappe, dato, radius=None, senterpunkt=(8.0182, 58.1599), antall_skip=10):
    """
    Filtrerer skip i Agder-området, enten basert på et definert polygon eller en radius fra et senterpunkt.

    Args:
        rot_mappe: Mappen hvor de konverterte dataene er lagret
        dato: Datoen å filtrere på (datetime objekt)
        radius: Radius i meter fra senterpunkt (None = bruk polygon)
        senterpunkt: Tuple (longitude, latitude) for senterpunkt (standard: Kristiansand)
        antall_skip: Maksimalt antall skip å returnere

    Returns:
        GeoDataFrame med filtrerte skip
    """
    # Først hent alle skip for datoen uten geografisk filtrering
    ships_on_date = filter_ais_data(
        root_folder=rot_mappe,
        date=dato,
        filters=None,
        agder_polygon=None,  # Ingen geografisk filter her ennå
        max_number_of_ships=100  # Hent litt flere for å kunne filtrere geografisk etterpå
    )

    if ships_on_date is None or ships_on_date.empty:
        print(f"Ingen skip funnet på datoen {dato.strftime('%d.%m.%Y')}")
        return None

    # Hvis radius er oppgitt, bruk finn_skip_rundt_punkt
    if radius is not None and radius > 0:
        ship_around_point, buffer = find_ships_near_point(
            ships_on_date,
            senterpunkt,
            buffer_distance=radius
        )

        # Begrens antall skip hvis nødvendig
        if ship_around_point is not None and len(ship_around_point) > antall_skip:
            ship_around_point = ship_around_point.head(antall_skip)

        if ship_around_point is None or ship_around_point.empty:
            print(f"Ingen skip funnet innenfor {radius}m fra senterpunkt på datoen {dato.strftime('%d.%m.%Y')}")
            return None

        #print(f"Fant {len(ship_around_point)} skip innenfor {radius}m på datoen {dato.strftime('%d.%m.%Y')}")
        return ship_around_point
    else:
        # Bruk standard Agder-polygon
        agder_polygon = define_agder_polygon()

        # Filtrer med polygon
        ships_in_range = ships_on_date[ships_on_date.geometry.within(agder_polygon)]

        # Begrens antall skip
        if len(ships_in_range) > antall_skip:
            ships_in_range = ships_in_range.head(antall_skip)

        if ships_in_range.empty:
            print(f"Ingen skip funnet innenfor Agder-område på datoen {dato.strftime('%d.%m.%Y')}")
            return None

        #print(f"Fant {len(ships_in_range)} skip innenfor Agder-område på datoen {dato.strftime('%d.%m.%Y')}")
        return ships_in_range

def visualize_ship_around_point(skip_gdf, point, buffer_distance=500, agder_polygon=None):
    """
    Visualiserer skip innenfor en buffer rundt et gitt punkt.

    Args:
        skip_gdf: GeoDataFrame med skip
        point: Tuple (longitude, latitude)
        buffer_distance: Avstand i meter
        agder_polygon: GeoDataFrame eller Shapely polygon med Agder-området (valgfri)

    Returns:
        Folium Map objekt
    """
    # Finn skip innenfor buffer
    ships_within_agder, buffer_wgs84 = find_ships_near_point(skip_gdf, point, buffer_distance)

    # Bestem senterpunkt for kartet (lat, lon)
    center_point = [point[1], point[0]]

    # Opprett kart
    m = folium.Map(
        location=center_point,
        zoom_start=9,
        tiles='CartoDB positron',
        control_scale=True
    )

    # Legg til flere bakgrunnskart
    folium.TileLayer('OpenStreetMap').add_to(m)

    if ships_within_agder is None or ships_within_agder.empty:
        print(f"Ingen skip funnet innenfor {buffer_distance}m fra punktet {point}")
    else:
        # Visualiser skip på kartet
        for idx, row in ships_within_agder.iterrows():
            # Anta at geopandas GeoDataFrame har en geometri-kolonne med Point-objekter
            ship_point = row.geometry
            ship_lat = ship_point.y
            ship_lon = ship_point.x

            # Lagre skip-info for popup
            popup_info = f"Skip ID: {row.get('mmsi', 'N/A')}<br>"
            if 'ship_name' in row:
                popup_info += f"Navn: {row['ship_name']}<br>"
            if 'ship_type' in row:
                popup_info += f"Type: {row['ship_type']}<br>"

            # Legg til markør for skipet
            folium.Marker(
                location=[ship_lat, ship_lon],
                popup=folium.Popup(popup_info, max_width=300),
                tooltip=row.get('ship_name', f"Skip {row.get('mmsi', idx)}"),
                icon=folium.Icon(icon='ship', prefix='fa', color='blue')
            ).add_to(m)

    # Legg til markør for senterpunkt
    folium.Marker(
        location=[point[1], point[0]],
        popup=f"Koordinater: {point}",
        tooltip="Senterpunkt",
        icon=folium.Icon(icon='crosshairs', prefix='fa', color='red')
    ).add_to(m)

    # Legg til sirkel for buffer
    folium.Circle(
        location=[point[1], point[0]],
        radius=buffer_distance,
        color='red',
        fill=True,
        fill_color='#ff7777',
        fill_opacity=0.2,
        popup=f"Buffer: {buffer_distance}m"
    ).add_to(m)

    # Hvis Agder-polygon er gitt, legg dette til kartet
    if agder_polygon is not None:
        # Konverter Shapely polygon til GeoJSON
        from shapely.geometry import mapping

        if hasattr(agder_polygon, "__geo_interface__"):
            # Hvis det allerede er et GeoJSON-kompatibelt objekt
            geo_json = agder_polygon.__geo_interface__
        else:
            # Ellers, konverter Shapely polygon til GeoJSON
            geo_json = mapping(agder_polygon)

        # Legg til polygon på kartet
        folium.GeoJson(
            data=geo_json,
            style_function=lambda x: {
                'fillColor': '#ffff00',
                'color': 'orange',
                'weight': 2,
                'fillOpacity': 0.1
            },
            name="Agder-område"
        ).add_to(m)

    # Legg til fullskjermsknapp og lagkontroll
    plugins.Fullscreen().add_to(m)
    folium.LayerControl().add_to(m)

    return m

def interactive_ship_search():
    """
    Oppretter et interaktivt grensesnitt for å søke etter skip med widgets
    med tabellvisning og sikrer at kartet vises etter tabellen
    """
    # Tekstfelt for manuell dato-inntasting
    date_input = widgets.Text(
        value='2025-01-05',
        placeholder='YYYY-MM-DD',
        description='Dato:',
        disabled=False
    )

    radius_slider = widgets.IntSlider(
        value=50,
        min=1,
        max=100,
        step=1,
        description='Radius (km):',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d'
    )

    search_button = widgets.Button(
        description='Søk etter skip',
        disabled=False,
        button_style='success',
        tooltip='Klikk for å søke',
        icon='search'
    )

    output_area = widgets.Output()

    # Lag layout
    display(widgets.VBox([
        widgets.HTML(value="<h3 style='text-align: left;'>Søk etter skip i Agder-området</h3>"),
        date_input,
        radius_slider,
        search_button,
        output_area
    ]))

    # Definer søkefunksjon
    def on_search_button_click(b):
        with output_area:
            clear_output(wait=True)

            date_str = date_input.value

            try:
                # Parse datoen
                search_date = datetime.strptime(date_str, '%Y-%m-%d')
                print(f"Søker etter skip registerert {date_str} med radius {radius_slider.value} km fra Kristiansand sentrum ...")

                # Konverter radius til meter
                radius_meters = radius_slider.value * 1000

                # Utfør søk med eksisterende funksjoner
                ships = filter_ships(
                    root_folder="data/processed",
                    dato=search_date,
                    radius=radius_meters,
                    center_point=(8.0, 58.15)
                )

                if ships is not None and not ships.empty:
                    print(f"Fant {len(ships)} skip.")
                    print("\nOppretter tabell ...")

                    # Vis skipene på tabellform
                    columns = ['mmsi', 'ship_name', 'ship_type', 'longitude', 'latitude']

                    # Velg bare kolonner som faktisk finnes i datasettet
                    available_columns = [kol for kol in columns if kol in ships.columns]

                    styled_tabell = ships[available_columns].style\
                        .format({'longitude': '{:.4f}', 'latitude': '{:.4f}'})\
                        .set_caption('Registrerte:')\
                        .set_table_styles([
                            {'selector': 'th', 'props': [('background-color', '#f2f2f2'),
                                                        ('color', '#333'),
                                                        ('font-weight', 'bold')]},
                            {'selector': 'td', 'props': [('padding', '5px')]}
                        ])

                    display(styled_tabell)

                    # Legg til litt plass mellom tabellen og kartet
                    #display(HTML("<div style='margin: 20px 0;'></div>"))
                    display(HTML("<style>.folium-map { width: 100%; height: 600px; }</style>"))

                    # Sikre at tabellen er ferdig rendret før kartet vises
                    time.sleep(0.5)

                    # Lag kartet og vis det - sikre at det blir rendret
                    try:
                        # Opprett og vis kartet
                        m = visualize_ship_around_point(
                            ships,
                            point=(8.0, 58.15),
                            buffer_distance=radius_meters,
                            agder_polygon=None  # Fjernet Agder-polygon visualisering
                        )

                        time.sleep(1)

                        # Gi eksplisitt beskjed om at kartet vises
                        print("Viser kart ...")

                        # Vis kartet
                        display(m)

                    except Exception as e:
                        print(f"Feil ved visualisering av kart: {e}")
                        print("Prøv å kjøre cellen på nytt hvis kartet ikke vises.")
                else:
                    print("Ingen skip funnet med disse søkekriteriene.")
            except ValueError as e:
                print(f"Ugyldig datoformat. Vennligst bruk formatet YYYY-MM-DD. Feil: {e}")

    # Koble til hendelse
    search_button.on_click(on_search_button_click)

## 4. Opprett mappestruktur

Kjør cellen nedenfor for å opprette nødvendige mapper:

In [ ]:
global_paths = create_folders()

## 5. Start konvertering og partisjonering

Nå er alt klart! Kjør cellen nedenfor for å starte konvertering og partisjonering.

Merk: Ønsker du å endre eller legge til partisjoneringskolonne(r) gjøres det i cellen nedenfor i "partition_columns".

In [39]:
# Manuell konvertering og partisjonering
results = start_conversion(data_folder="./data", partition_columns=["date_time_utc", "ship_type"])

Starter konvertering og partisjonering av filer ...

Valgte kolonner: ['date_time_utc', 'ship_type']
Vent litt ...

Konvertering og partisjonering utført:
• Total behandlingstid: 5.04 sekunder
• Konverterte filer: 9


## 6. Hent og visualiser skip
Legg inn egne verdier for dato og radius fra Kristiansand sentrum i feltene under og klikk "Søk etter skip".

In [34]:
interactive_ship_search()